In [2]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [4]:
data_df = pd.read_pickle('data_clean.pickle')
data_df

,text,subreddit
wsb,in the future when apple is at a share your g...,wsb
science,the children were not motivated by achieving ...,science
ama,tldr do whatever the fuck you want so youre te...,ama
askreddit,victorias secret was originally supposed to be...,askreddit
worldnews,so was there any posts left after the purge ok...,worldnews
funny,but i wanted the robot ferret haha suck it you...,funny
dankmemes,meme of the month seeding and nominations for ...,dankmemes
memes,redditmc is opening staff positions builders h...,memes
nosleep,it looks like there may be more to this story ...,nosleep
psychology,decisions regarding mental health care canada...,psychology


In [5]:
wsb = data_df[:1]
wsb.text

wsb    in the future when apple is at  a share your g...
Name: text, dtype: object

In [7]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = TfidfVectorizer(ngram_range=(1,2))
data_cvn = cvn.fit_transform(wsb.text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# data_dtmn.columns = data_df.index
data_dtmn

,0
aaa,0.000158
aaa is,0.000032
aaa literally,0.000032
aaa loans,0.000032
aaa video,0.000032
...,...
𝓭𝓸𝔀𝓷 im,0.000032
𝕵𝖊𝖘𝖚𝖘,0.000032
𝕵𝖊𝖘𝖚𝖘 𝕷𝖊𝖆𝖉𝖊𝖗𝖇𝖔𝖆𝖗𝖉,0.000032
𝕷𝖊𝖆𝖉𝖊𝖗𝖇𝖔𝖆𝖗𝖉,0.000032


In [4]:
# data_dtmn['index'] = data_dtmn.index
# for i, row in enumerate(data_dtmn['index']):
#     candidate = re.match('a{2,}', str(row))
#     if candidate != None:
#         data_dtmn.drop(row, inplace=True)
#     if row == 'ababababba':
#         data_dtmn.drop(row, inplace=True)
#     if row == 'wa':
#         data_dtmn.drop(row, inplace=True)       
#     if row == 'ha':
#         data_dtmn.drop(row, inplace=True)    
# data_dtmn.drop(columns=['index'], inplace=True)
# data_dtmn

In [8]:
from gensim import matutils, models
import scipy.sparse

In [9]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [10]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=10, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 15:30:03: using symmetric alpha at 0.1
INFO - 15:30:03: using symmetric eta at 0.1
INFO - 15:30:04: using serial LDA version on this node
INFO - 15:30:04: running online LDA training, 10 topics, 50 passes over the supplied corpus of 228409 documents, updating every 30000 documents, evaluating every ~228409 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 15:30:04: training LDA model using 15 processes
INFO - 15:30:04: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/228409, outstanding queue size 1
INFO - 15:30:04: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/228409, outstanding queue size 2
INFO - 15:30:04: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/228409, outstanding queue size 3
INFO - 15:30:04: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/228409, outstanding queue size 4
INFO - 15:30:04: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/228409, outstanding queue size 5
INF

INFO - 15:30:06: topic #1 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:30:06: topic #5 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:30:06: topic diff=0.000002, rho=0.250000
INFO - 15:30:06: PROGRESS: pass 0, dispatched chunk #50 = documents up to #102000/228409, outstanding queue size 21
INFO - 15:30:06: PROGRESS: pass 0, dispatched chunk #51 = documents up to #104000/228409, outstanding queue size 16
INFO - 15:30:07: PROGRESS: pass 0, dispatched chunk #52 = documents up to #106000/228409, outstanding queue size 16
INFO - 15:30:07: PROGRESS: pass 0, dispatched chunk #53 = documents up to #108000/228409, outstanding queue size 16
INF

INFO - 15:30:09: PROGRESS: pass 0, dispatched chunk #82 = documents up to #166000/228409, outstanding queue size 3
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #83 = documents up to #168000/228409, outstanding queue size 4
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #84 = documents up to #170000/228409, outstanding queue size 5
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #85 = documents up to #172000/228409, outstanding queue size 6
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #86 = documents up to #174000/228409, outstanding queue size 6
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #87 = documents up to #176000/228409, outstanding queue size 6
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #88 = documents up to #178000/228409, outstanding queue size 6
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #89 = documents up to #180000/228409, outstanding queue size 6
INFO - 15:30:10: PROGRESS: pass 0, dispatched chunk #90 = documents up to #18200

INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #2 = documents up to #6000/228409, outstanding queue size 3
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #3 = documents up to #8000/228409, outstanding queue size 4
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #4 = documents up to #10000/228409, outstanding queue size 5
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #5 = documents up to #12000/228409, outstanding queue size 6
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #6 = documents up to #14000/228409, outstanding queue size 7
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #7 = documents up to #16000/228409, outstanding queue size 8
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #8 = documents up to #18000/228409, outstanding queue size 9
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #9 = documents up to #20000/228409, outstanding queue size 10
INFO - 15:30:12: PROGRESS: pass 1, dispatched chunk #10 = documents up to #22000/228409, outstand

INFO - 15:30:15: PROGRESS: pass 1, dispatched chunk #51 = documents up to #104000/228409, outstanding queue size 11
INFO - 15:30:15: PROGRESS: pass 1, dispatched chunk #52 = documents up to #106000/228409, outstanding queue size 11
INFO - 15:30:15: PROGRESS: pass 1, dispatched chunk #53 = documents up to #108000/228409, outstanding queue size 11
INFO - 15:30:15: PROGRESS: pass 1, dispatched chunk #54 = documents up to #110000/228409, outstanding queue size 11
INFO - 15:30:15: merging changes from 30000 documents into a model of 228409 documents
INFO - 15:30:16: topic #1 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:30:16: topic #4 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"p

...

INFO - 15:36:50: topic #7 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:50: topic #5 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:50: topic #8 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:50: topic #4 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:50: topic #2 (0.100): 0

INFO - 15:36:52: PROGRESS: pass 49, dispatched chunk #34 = documents up to #70000/228409, outstanding queue size 6
INFO - 15:36:53: merging changes from 30000 documents into a model of 228409 documents
INFO - 15:36:53: topic #1 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:53: topic #2 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:53: topic #5 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:53: topic #9 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"

INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #71 = documents up to #144000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #72 = documents up to #146000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #73 = documents up to #148000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #74 = documents up to #150000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #75 = documents up to #152000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #76 = documents up to #154000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #77 = documents up to #156000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #78 = documents up to #158000/228409, outstanding queue size 8
INFO - 15:36:55: PROGRESS: pass 49, dispatched chunk #79 = documents up 

INFO - 15:36:58: topic #4 (0.100): 0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"
INFO - 15:36:58: topic diff=0.000000, rho=0.078038
INFO - 15:36:58: PROGRESS: pass 49, dispatched chunk #110 = documents up to #222000/228409, outstanding queue size 6
INFO - 15:36:58: PROGRESS: pass 49, dispatched chunk #111 = documents up to #224000/228409, outstanding queue size 3
INFO - 15:36:58: PROGRESS: pass 49, dispatched chunk #112 = documents up to #226000/228409, outstanding queue size 4
INFO - 15:36:58: PROGRESS: pass 49, dispatched chunk #113 = documents up to #228000/228409, outstanding queue size 5
INFO - 15:36:58: PROGRESS: pass 49, dispatched chunk #114 = documents up to #228409/228409, outstanding queue size 6
INFO - 15:36:58: merging changes from 18409 documents into a model of 228409 documents
INFO - 15:36:58: topic #7 (0.100): 0

[(0,
  '0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"'),
 (1,
  '0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"'),
 (2,
  '0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"'),
 (3,
  '0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"pumpers or" + 0.000*"pumpernickle hold" + 0.000*"pumpers as" + 0.000*"pumping all"'),
 (4,
  '0.000*"aaa" + 0.000*"pumpfake" + 0.000*"pumpin we" + 0.000*"pumpin" + 0.000*"pumpfake to" + 0.000*"pumpfake like" + 0.000*"p